In [1]:
#All works at the MET - Federico Zeri Catalog

from SPARQLWrapper import SPARQLWrapper, JSON
import sparql_dataframe
import pandas as pd

endpoint = "http://data.fondazionezeri.unibo.it/sparql"

query = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX ti: <http://www.ontologydesignpatterns.org/cp/owl/timeinterval.owl#>
SELECT DISTINCT ?artwork ?label_artwork ?inventory
WHERE {
  ?artwork crm:P50_has_current_keeper <https://w3id.org/zericatalog/the-metropolitan-museum-of-art-new-york-ny> .
  ?artwork  rdfs:label ?label_artwork.
  ?artwork crm:P55_has_current_location ?location .
  ?location  crm:P3_has_note ?inventory . 
  FILTER(LANG(?label_artwork) = "" || LANGMATCHES(LANG(?label_artwork), "en"))

}
"""

df = sparql_dataframe.get(endpoint, query, post=True)
df

,artwork,label_artwork,inventory
0,https://w3id.org/zericatalog/artwork/18789/item,"Artwork ""Luciani Sebastiano , Ritratto di Cris...",inv. 00.18.2
1,https://w3id.org/zericatalog/artwork/16633/item,"Artwork ""Del Conte Jacopino , Sacra Famiglia"" ...","inv. 1976.100.15, lascito Harry G. Sperling"
2,https://w3id.org/zericatalog/artwork/18917/item,"Artwork ""Anonimo sec. XVI , Ritratto di monaco...",inv. 1986.339.1
3,https://w3id.org/zericatalog/artwork/18008/item,"Artwork ""Rossi Francesco de' , Ritratto maschi...",inv. 45.128.11
4,https://w3id.org/zericatalog/artwork/18022/item,"Artwork ""Rossi Francesco de' , Ritratto maschi...",inv. 55.14
...,...,...,...
103,https://w3id.org/zericatalog/artwork/50670/item,"Artwork ""Bonvicino Alessandro , Ritratto di ge...",inv. 28.79
104,https://w3id.org/zericatalog/artwork/49125/item,"Artwork ""Tisi Benvenuto , San Nicola da Tolent...",inv. 17.190.24
105,https://w3id.org/zericatalog/artwork/49129/item,"Artwork ""Tisi Benvenuto , San Nicola da Tolent...",inv. 17.190.23
106,https://w3id.org/zericatalog/artwork/50819/item,"Artwork ""Bonvicino Alessandro , Deposizione di...",inv. 12.61


In [8]:
#Works' History - Federico Zeri Catalog

URI_string = ""

for index, row in df.iterrows():
    if len(URI_string) == 0:
        URI_string = "?artwork = <" + row["artwork"] + ">"
    else: 
        URI_string = URI_string + "||" + "?artwork = <" + row["artwork"] + ">"

final_query = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX ti: <http://www.ontologydesignpatterns.org/cp/owl/timeinterval.owl#>
SELECT ?artwork  (count(distinct ?trasferimento_global) AS ?history) (group_concat(distinct ?label_receiver ; separator = ", ") AS ?receivers) (group_concat(distinct ?inventory ; separator = ", ") AS ?id) (group_concat(distinct ?label_surrender ; separator = ", ") AS ?surrenders)          
WHERE {{
  ?trasferimento_global crm:P30_transferred_custody_of ?artwork .
  ?trasferimento_global crm:P28_custody_surrendered_by ?surrender .
  ?receiver crm:P29i_received_custody_through ?trasferimento_global .
  ?artwork crm:P50_has_current_keeper <https://w3id.org/zericatalog/the-metropolitan-museum-of-art-new-york-ny> .
  ?artwork crm:P55_has_current_location ?location .
  ?location  crm:P3_has_note ?inventory .
  FILTER ({0})
  
  ?artwork  rdfs:label ?label_artwork.
  ?trasferimento_global rdfs:label ?label_transfer.
  ?surrender rdfs:label ?label_surrender.
  ?receiver rdfs:label ?label_receiver.
  
  FILTER(?surrender != <zeri:https://w3id.org/zericatalog/>)
  
  FILTER(LANG(?label_artwork) = "" || LANGMATCHES(LANG(?label_artwork), "en"))
  FILTER(LANG(?label_transfer) = "" || LANGMATCHES(LANG(?label_transfer), "en"))
  FILTER(LANG(?label_surrender) = "" || LANGMATCHES(LANG(?label_surrender), "en"))
  FILTER(LANG(?label_receiver) = "" || LANGMATCHES(LANG(?label_receiver), "en"))
}} GROUP BY ?artwork
"""

final_df = sparql_dataframe.get(endpoint, final_query.format(URI_string), post=True)
final_df

,artwork,history,receivers,id,surrenders
0,https://w3id.org/zericatalog/artwork/39117/item,6,"The Metropolitan Museum of Art, Asta Sotheby's...",inv. 1970.134.1,"Asta Sotheby's, Asta Christie's, Collezione L...."
1,https://w3id.org/zericatalog/artwork/39828/item,5,"The Metropolitan Museum of Art, E. Sestieri, C...",inv. 48.17.12,"E. Sestieri, Collezione Malanca, Collezione G...."
2,https://w3id.org/zericatalog/artwork/39818/item,5,"The Metropolitan Museum of Art, E. Sestieri, C...",inv. 48.17.5,"E. Sestieri, Collezione Malanca, Collezione G...."
3,https://w3id.org/zericatalog/artwork/45055/item,1,The Metropolitan Museum of Art,"inv. 14.40.650, dono B. Altman",Collezione B. Altman
4,https://w3id.org/zericatalog/artwork/39821/item,5,"The Metropolitan Museum of Art, E. Sestieri, C...",inv. 48.17.7,"E. Sestieri, Collezione Malanca, Collezione G...."
5,https://w3id.org/zericatalog/artwork/39831/item,5,"The Metropolitan Museum of Art, E. Sestieri, C...",inv. 48.17.14,"E. Sestieri, Collezione Malanca, Collezione G...."
6,https://w3id.org/zericatalog/artwork/16633/item,4,"The Metropolitan Museum of Art, Collezione H.G...","inv. 1976.100.15, lascito Harry G. Sperling","Collezione H.G. Sperling, Kleinberger Gallerie..."
7,https://w3id.org/zericatalog/artwork/41423/item,2,"The Metropolitan Museum of Art, Asta Christie's",inv. 1973.311.1,"Asta Christie's, Harewood House, collezione Ea..."
8,https://w3id.org/zericatalog/artwork/60085/item,1,The Metropolitan Museum of Art,inv. 1971.115.3b,"Duveen Brothers, Inc."
9,https://w3id.org/zericatalog/artwork/39841/item,5,"The Metropolitan Museum of Art, E. Sestieri, C...",inv. 48.17.20,"E. Sestieri, Collezione Malanca, Collezione G...."


In [18]:
a_list = []

for idx, row in final_df.iterrows():
    a_list.append(str(row["id"]).replace("inv. ", "").split(",")[0])
a_list
final_df["inventory"] = a_list
final_df = final_df.drop(['id'], axis=1)

In [24]:
italianDataset = pd.read_csv("data.csv", keep_default_na=False, dtype={'AccessionYear':int})
italianDataset = italianDataset.iloc[:,1:]
italianDataset

,Object Number,Is Highlight,Is Timeline Work,Is Public Domain,Object ID,Gallery Number,Department,AccessionYear,Object Name,Title,...,River,Classification,Rights and Reproduction,Link Resource,Object Wikidata URL,Metadata Date,Repository,Tags,Tags AAT URL,Tags Wikidata URL
0,1985.5,0,0,1,435573,,European Paintings,1985,Painting,Flora and Zephyr,...,,Paintings,,http://www.metmuseum.org/art/collection/search...,https://www.wikidata.org/wiki/Q19912099,,"Metropolitan Museum of Art, New York, NY",Goddess|Putti|Flowers|Landscapes,http://vocab.getty.edu/page/aat/300343852|http...,https://www.wikidata.org/wiki/Q205985|https://...
1,12.6,0,0,1,435574,,European Paintings,1912,"Painting, predella panel",The Crucifixion,...,,Paintings,,http://www.metmuseum.org/art/collection/search...,https://www.wikidata.org/wiki/Q19905919,,"Metropolitan Museum of Art, New York, NY",Soldiers|Men|Crucifixion|Horses|Mountains|Ange...,http://vocab.getty.edu/page/aat/300185678|http...,https://www.wikidata.org/wiki/Q4991371|https:/...
2,42.53.2,0,0,1,435575,,European Paintings,1942,"Painting, miniature","Jérôme Bonaparte (1784–1860), King of Westphalia",...,,Miniatures,,http://www.metmuseum.org/art/collection/search...,https://www.wikidata.org/wiki/Q19913805,,"Metropolitan Museum of Art, New York, NY",Kings|Men|Portraits,http://vocab.getty.edu/page/aat/300025481|http...,https://www.wikidata.org/wiki/Q12097|https://w...
3,1991.27.2,0,1,1,435576,624.0,European Paintings,1991,Painting,Saint Alexander,...,,Paintings,,http://www.metmuseum.org/art/collection/search...,https://www.wikidata.org/wiki/Q19912513,,"Metropolitan Museum of Art, New York, NY",Bishops|Croziers|Miter,http://vocab.getty.edu/page/aat/300025756|http...,https://www.wikidata.org/wiki/Q29182|https://w...
4,14.40.628,0,1,1,435577,,European Paintings,1914,Painting,The Crucifixion,...,,Paintings,,http://www.metmuseum.org/art/collection/search...,https://www.wikidata.org/wiki/Q19905200,,"Metropolitan Museum of Art, New York, NY",Crucifixion|Saints|Mary Magdalene|Saint Franci...,http://vocab.getty.edu/page/aat/300404300|http...,https://www.wikidata.org/wiki/Q3235597|https:/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,2014.277,0,0,1,643540,617.0,European Paintings,2014,Painting,Portrait of a Woman,...,,Paintings,,http://www.metmuseum.org/art/collection/search...,https://www.wikidata.org/wiki/Q19911984,,"Metropolitan Museum of Art, New York, NY",Portraits|Women,http://vocab.getty.edu/page/aat/300015637|http...,https://www.wikidata.org/wiki/Q134307|https://...
478,2014.270,0,0,1,647338,601.0,European Paintings,2014,Painting,Saint Francis in Ecstasy,...,,Paintings,,http://www.metmuseum.org/art/collection/search...,https://www.wikidata.org/wiki/Q19911615,,"Metropolitan Museum of Art, New York, NY",Skulls|Saint Francis|Cross,http://vocab.getty.edu/page/aat/300191856|http...,https://www.wikidata.org/wiki/Q13147|https://w...
479,2014.448,0,0,1,657683,608.0,European Paintings,2014,Painting,The Lamentation,...,,Paintings,,http://www.metmuseum.org/art/collection/search...,https://www.wikidata.org/wiki/Q19911616,,"Metropolitan Museum of Art, New York, NY",Men|Women|Lamentation|Christ|Saint John the Ev...,http://vocab.getty.edu/page/aat/300025928|http...,https://www.wikidata.org/wiki/Q8441|https://ww...
480,2014.566,0,0,1,659875,622.0,European Paintings,2014,Painting,The Execution of Saint John the Baptist,...,,Paintings,,http://www.metmuseum.org/art/collection/search...,https://www.wikidata.org/wiki/Q19911986,,"Metropolitan Museum of Art, New York, NY",Swords|Saint John the Baptist|Execution|Men|Go...,http://vocab.getty.edu/page/aat/300037048|http...,https://www.wikidata.org/wiki/Q12791|https://w...


In [26]:
#merging MET and Federico Zeri

from pandas import merge

result = merge(final_df, italianDataset, left_on="inventory", right_on="Object Number", how="inner")
result
result.to_csv("result.csv")

open_df = pd.read_csv("result.csv")
open_df

,Unnamed: 0,artwork,history,receivers,surrenders,inventory,Object Number,Is Highlight,Is Timeline Work,Is Public Domain,...,River,Classification,Rights and Reproduction,Link Resource,Object Wikidata URL,Metadata Date,Repository,Tags,Tags AAT URL,Tags Wikidata URL
0,0,https://w3id.org/zericatalog/artwork/39117/item,6,"The Metropolitan Museum of Art, Asta Sotheby's...","Asta Sotheby's, Asta Christie's, Collezione L....",1970.134.1,1970.134.1,0,0,1,...,NaN,Paintings,NaN,http://www.metmuseum.org/art/collection/search...,https://www.wikidata.org/wiki/Q19911541,NaN,"Metropolitan Museum of Art, New York, NY",Infants|Women|Saint John the Baptist|Bedrooms|...,http://vocab.getty.edu/page/aat/300189561|http...,https://www.wikidata.org/wiki/Q998|https://www...
1,1,https://w3id.org/zericatalog/artwork/45055/item,1,The Metropolitan Museum of Art,Collezione B. Altman,14.40.650,14.40.650,0,1,1,...,NaN,Paintings,NaN,http://www.metmuseum.org/art/collection/search...,https://www.wikidata.org/wiki/Q19904807,NaN,"Metropolitan Museum of Art, New York, NY",Men|Portraits,http://vocab.getty.edu/page/aat/300025928|http...,https://www.wikidata.org/wiki/Q8441|https://ww...
2,2,https://w3id.org/zericatalog/artwork/16633/item,4,"The Metropolitan Museum of Art, Collezione H.G...","Collezione H.G. Sperling, Kleinberger Gallerie...",1976.100.15,1976.100.15,0,0,1,...,NaN,Paintings,NaN,http://www.metmuseum.org/art/collection/search...,https://www.wikidata.org/wiki/Q19912153,NaN,"Metropolitan Museum of Art, New York, NY",Holy Family|Saint Joseph|Christ|Virgin Mary,http://vocab.getty.edu/page/ia/901001175|http:...,https://www.wikidata.org/wiki/Q618057|https://...
3,3,https://w3id.org/zericatalog/artwork/41423/item,2,"The Metropolitan Museum of Art, Asta Christie's","Asta Christie's, Harewood House, collezione Ea...",1973.311.1,1973.311.1,0,1,1,...,NaN,Paintings,NaN,http://www.metmuseum.org/art/collection/search...,https://www.wikidata.org/wiki/Q19905185,NaN,"Metropolitan Museum of Art, New York, NY",Armor|Boys|Men|Portraits,http://vocab.getty.edu/page/aat/300226591|http...,https://www.wikidata.org/wiki/Q20793164|https:...
4,4,https://w3id.org/zericatalog/artwork/32096/item,4,"The Metropolitan Museum of Art, P. Corsini, Co...","P. Corsini, Collezione S. Brunswick, Hammer Ga...",1984.74,1984.74,0,1,1,...,NaN,Paintings,NaN,http://www.metmuseum.org/art/collection/search...,https://www.wikidata.org/wiki/Q19911628,NaN,"Metropolitan Museum of Art, New York, NY",Sadness|Suffering|Mary Magdalene|Saint John th...,http://vocab.getty.edu/page/aat/300055166|http...,https://www.wikidata.org/wiki/Q169251|https://...
5,5,https://w3id.org/zericatalog/artwork/45062/item,4,"The Metropolitan Museum of Art, Collezione J. ...","Collezione J. Pierpont Morgan, Collezioni real...",16.30ab,16.30ab,1,1,1,...,NaN,Paintings,NaN,http://www.metmuseum.org/art/collection/search...,https://www.wikidata.org/wiki/Q27033728,NaN,"Metropolitan Museum of Art, New York, NY",Madonna and Child|Saint Catherine|Saint John t...,http://vocab.getty.edu/page/ia/901000052| |htt...,https://www.wikidata.org/wiki/Q9309699| |https...
6,6,https://w3id.org/zericatalog/artwork/46494/item,1,The Metropolitan Museum of Art,Collezione G. e F. Blumenthal,41.100.12,41.100.12,0,1,1,...,NaN,Paintings,NaN,http://www.metmuseum.org/art/collection/search...,https://www.wikidata.org/wiki/Q19905075,NaN,"Metropolitan Museum of Art, New York, NY",Men|Portraits,http://vocab.getty.edu/page/aat/300025928|http...,https://www.wikidata.org/wiki/Q8441|https://ww...
7,7,https://w3id.org/zericatalog/artwork/18022/item,3,"The Metropolitan Museum of Art, Collezione J.H...","Collezione J.H. Harding, Asta Christie's, Coll...",55.14,55.14,0,1,1,...,NaN,Paintings,NaN,http://www.metmuseum.org/art/collection/search...,https://www.wikidata.org/wiki/Q19912783,NaN,"Metropolitan Museum of Art, New York, NY",Men|Portraits,http://vocab.getty.edu/page/aat/300025928|http...,https://www.wikidata.org/wiki/Q8441|https://ww...
8,8,https://w3id.org/zericatalog/art

In [28]:
import re

new_string = ""
for idx, row in open_df.iterrows():
    if idx == 0:
        new_string = new_string + "strstarts(str(?custody), '" + row["artwork"].replace("/item", "/custody") + "')"
    else:
        new_string = new_string + "|| strstarts(str(?custody), '" + row["artwork"].replace("/item", "/custody") + "')"


access = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX ti: <http://www.ontologydesignpatterns.org/cp/owl/timeinterval.owl#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
SELECT DISTINCT ?custody ?time_span ?start_date ?end_date ?place_label ?label_receiver ?location
WHERE {{
  ?receiver crm:P29i_received_custody_through ?custody .
  ?receiver crm:P74_has_current_or_former_residence ?place .
  ?custody crm:P4_has_time_span ?time_span .
  ?time_span ti:hasIntervalEndDate ?end_date .
  ?time_span ti:hasIntervalStartDate ?start_date .
  ?place owl:sameAs ?location .
  ?place rdfs:label ?place_label .
  ?receiver rdfs:label ?label_receiver .
  FILTER(LANG(?place_label) = "" || LANGMATCHES(LANG(?place_label), "en"))
  FILTER(LANG(?label_receiver) = "" || LANGMATCHES(LANG(?label_receiver), "en"))
  FILTER ({0})
}}
"""
third_df = sparql_dataframe.get(endpoint, access.format(new_string), post=True)   
third_df


,custody,time_span,start_date,end_date,place_label,label_receiver,location
0,https://w3id.org/zericatalog/artwork/16633/cus...,https://w3id.org/zericatalog/date/1949-1958,1949-01-01T00:00:00Z,1958-01-01T00:00:00Z,Parigi,Collezione O. Wertheimer,http://sws.geonames.org/2988507
1,https://w3id.org/zericatalog/artwork/16633/cus...,https://w3id.org/zericatalog/date/1949-1958,1949-01-01T00:00:00Z,1958-01-01T00:00:00Z,Parigi,Collezione O. Wertheimer,http://www.wikidata.org/entity/Q90
2,https://w3id.org/zericatalog/artwork/16633/cus...,https://w3id.org/zericatalog/date/1949-1958,1949-01-01T00:00:00Z,1958-01-01T00:00:00Z,Parigi,Collezione O. Wertheimer,http://dbpedia.org/resource/Paris
3,https://w3id.org/zericatalog/artwork/45062/cus...,https://w3id.org/zericatalog/date/1678-1798,1678-01-01T00:00:00Z,1798-01-01T00:00:00Z,Roma,Collezione Colonna,http://sws.geonames.org/3169069
4,https://w3id.org/zericatalog/artwork/45062/cus...,https://w3id.org/zericatalog/date/1678-1798,1678-01-01T00:00:00Z,1798-01-01T00:00:00Z,Roma,Collezione Colonna,http://www.wikidata.org/entity/Q15119
5,https://w3id.org/zericatalog/artwork/45062/cus...,https://w3id.org/zericatalog/date/1678-1798,1678-01-01T00:00:00Z,1798-01-01T00:00:00Z,Roma,Collezione Colonna,http://dbpedia.org/resource/Province_of_Rome
